# SectorScan

We will build a financial analytics app, called <b>SectorScan</b>, that turns data from different sectors into beautiful, easy-to-understand visualizations. User can choose which sectors to compare, and the app will analyze the Market Cap, Valuation, and Top Companies in those sectors.


Why do we want to build this app?

* By comparing various sectors, user can quickly spot which ones are performing better than the others. It’s like having a bird’s eye view of the market, making it easier to see trends and patterns that might not be obvious otherwise.

* This app is great for investors who want to track the market without getting lost in the details.

* And the last but not least, effort saving! All of the data is served in only one page. No need to open up many pages just to compare data from different sectors.

Explore it now: https://sectorscan.streamlit.app/

## Import Library & API Key

In [ ]:
from google.colab import userdata
api_key = userdata.get('sectors_api')

In [ ]:
import pandas as pd
import requests
import altair as alt

## Helper Functions

In [ ]:
def retrieve_from_endpoint(url:str):
  headers = {"Authorization": api_key}

  try:
      response = requests.get(url, headers=headers)
      response.raise_for_status()
      data = response.json()
  except requests.exceptions.HTTPError as err:
      raise SystemExit(err)
  return data

In [ ]:
def retrieve_sectors_data():
  url = "https://api.sectors.app/v1/subsectors/"
  sectors = retrieve_from_endpoint(url)
  sectors = [item['subsector'] for item in sectors]
  return sectors

In [ ]:
def retrieve_market_cap_data(sectors_list):
  # df to store total market cap
  df_mc_curr = pd.DataFrame()
  # df to store quarterly market cap
  df_mc_hist = pd.DataFrame()
  # df to store monthly performance
  df_mc_change = pd.DataFrame()

  for sector in sectors_list:
    url = f"https://api.sectors.app/v1/sector/report/{sector}/?sections=market_cap"
    market_cap = retrieve_from_endpoint(url)

    # Process df_mc_curr
    mc_curr = pd.DataFrame({
      "Sector": market_cap["sub_sector"],
      "Total Market Cap": market_cap["market_cap"]["total_market_cap"]
    }, index=[0])
    df_mc_curr = pd.concat([df_mc_curr, mc_curr], ignore_index=True)

    # Process df_mc_hist
    mc_hist = pd.DataFrame()
    for i in ["prev_ttm_mcap", "current_ttm_mcap"]:
      df = pd.melt(pd.DataFrame(market_cap["market_cap"]["quarterly_market_cap"][i], index=["quarter"]), var_name="Quarter", value_name="Market Cap")
      df["Sector"] = market_cap["sub_sector"]
      df = df[["Sector", "Quarter", "Market Cap"]]
      mc_hist = pd.concat([mc_hist, df], ignore_index=True)
    df_mc_hist = pd.concat([df_mc_hist, mc_hist], ignore_index=True)

    # Process df_mc_change
    mc_change = pd.melt(pd.DataFrame(market_cap["market_cap"]["mcap_summary"]["monthly_performance"], index=["date"]), var_name="Date", value_name="Market Cap Change")
    mc_change["Sector"] = market_cap["sub_sector"]
    mc_change = mc_change[["Sector", "Date", "Market Cap Change"]]
    df_mc_change = pd.concat([df_mc_change, mc_change], ignore_index=True)

  # Finalize df_mc_curr
  url = f"https://api.sectors.app/v1/sector/report/{sectors_list[0]}/?sections=idx"
  idx = retrieve_from_endpoint(url)

  idx_mc = idx["idx"]["idx_cap"]

  # Calculate the total market cap of the other sectors
  df_idx_mc = pd.DataFrame({
          "Sector": "Others",
          "Total Market Cap": idx_mc - df_mc_curr["Total Market Cap"].sum()
  }, index=[0])

  # Combine it with df_mc_curr
  df_mc_curr = pd.concat([df_mc_curr, df_idx_mc], ignore_index=True)

  # Calculate % market cap of each sector
  df_mc_curr["% Market Cap"] = (df_mc_curr["Total Market Cap"] / df_mc_curr["Total Market Cap"].sum()) * 100

  # Calculate market cap in trillion IDR
  df_mc_curr["Market Cap (Trillion IDR)"] = df_mc_curr["Total Market Cap"]/10**12

  # Finalize df_mc_hist
  df_mc_hist["Market Cap (Trillion IDR)"] = df_mc_hist["Market Cap"]/10**12

  # Finalize df_mc_change
  df_mc_change["Market Cap Change (%)"] = df_mc_change["Market Cap Change"] * 100

  return df_mc_curr, df_mc_hist, df_mc_change

In [ ]:
def retrieve_valuation_data(sectors_list):
  # df to store valuation data
  df_valuation = pd.DataFrame()

  for sector in sectors_list:
    url = f"https://api.sectors.app/v1/sector/report/{sector}/?sections=valuation"
    valuation = retrieve_from_endpoint(url)

    # Process df_valutation
    df = pd.DataFrame(valuation["valuation"]["historical_valuation"])
    df["Sector"] = valuation["sub_sector"]
    df_valuation = pd.concat([df_valuation, df], ignore_index=True)

  # Remove unused columns and rename columns
  try:
      df_valuation = df_valuation.drop(["pb_rank", "pe_rank", "ps_rank", "pcf_rank"], axis=1)
  except:
      pass
  df_valuation.columns = ["Price/Book Ratio", "Price/Earning Ratio", "Price/Sales Ratio", "Price/Cash Flow Ratio", "Year", "Sector"]

  return df_valuation

In [ ]:
def retrieve_top_companies_data(sectors_list):
  # df to store top companies based on market cap
  df_top_mc = pd.DataFrame()
  # df to store top companies based on revenue growth
  df_top_growth = pd.DataFrame()
  # df to store top companies based on profit
  df_top_profit = pd.DataFrame()
  # df to store top companies based on revenue
  df_top_revenue = pd.DataFrame()

  for sector in sectors_list:
    url = f"https://api.sectors.app/v1/sector/report/{sector}/?sections=companies"
    company = retrieve_from_endpoint(url)

    # Process df_top_mc, df_top_growth, df_top_profit, df_top_revenue
    keys = ["top_mcap", "top_growth", "top_profit", "top_revenue"]
    dfs = {}

    for key in keys:
        df = pd.DataFrame(company["companies"]["top_companies"][key])
        df["Sector"] = company["sub_sector"]
        dfs[key] = df

    df_top_mc = pd.concat([df_top_mc, dfs["top_mcap"]], ignore_index=True)
    df_top_growth = pd.concat([df_top_growth, dfs["top_growth"]], ignore_index=True)
    df_top_profit = pd.concat([df_top_profit, dfs["top_profit"]], ignore_index=True)
    df_top_revenue = pd.concat([df_top_revenue, dfs["top_revenue"]], ignore_index=True)

  # Rename columns
  df_top_mc.columns = ["Symbol", "Market Cap", "Sector"]
  df_top_growth.columns = ["Symbol", "Revenue Growth", "Sector"]
  df_top_profit.columns = ["Symbol", "Profit", "Sector"]
  df_top_revenue.columns = ["Symbol", "Revenue", "Sector"]

  # Finalize data frames
  df_top_mc["Market Cap (Trillion IDR)"] = df_top_mc["Market Cap"]/10**12
  df_top_growth["Revenue Growth (%)"] = df_top_growth["Revenue Growth"] * 100
  df_top_profit["Profit (Billion IDR)"] = df_top_profit["Profit"]/10**9
  df_top_revenue["Revenue (Trillion IDR)"] = df_top_revenue["Revenue"]/10**12

  return df_top_mc, df_top_growth, df_top_profit, df_top_revenue

## Dive into the Data

### Sectors

First thing first, let's get the list of the sectors from this [endpoint](https://docs.sectors.app/api-references/helper-list/subsectors). This list will be used as our filter list.

In [ ]:
# Call retrieve_sectors_data() function
sectors_list = retrieve_sectors_data()
sectors_list

All data that we're going to retrieved from this moment on is from this [endpoint](https://docs.sectors.app/api-references/report/sector). We just have to provide the section that we want to retrieved.

### Market Cap

Now let's retrieve the market cap data of the sectors. Since we're still in this notebook, we'll use the first 3 sectors. Later on in the app, we can use the selected sectors from the filter.

There are 3 data that we're going to retrieved:

* `total_mc` : total market cap data
* `historical_mc`: historical market cap data
* `change_mc`: historical market cap change data

In [ ]:
# Call retrieve_market_cap_data() function
total_mc, historical_mc, change_mc = retrieve_market_cap_data(sectors_list[0:3])

#### Total Market Cap

Let's compare the percentage of the sectors' total market cap compared to the total IDX market cap. So we know, which sectors dominate the market.

In [ ]:
# Snippet of the data
total_mc.head()

We'll use [pie chart](https://altair-viz.github.io/user_guide/marks/arc.html) from altair.

Altair color scheme: [check it here](https://vega.github.io/vega/docs/schemes/).

In [ ]:
total_mc_chart = alt.Chart(total_mc).mark_arc().encode(
    theta=alt.Theta("% Market Cap:Q"),
    color=alt.Color(
        "Sector:N",
        scale=alt.Scale(scheme="reds"),
        sort=alt.EncodingSortField(field="% Market Cap", order="ascending") # sort color based on % Market Cap value
    ),
    tooltip=[
        alt.Tooltip("Sector:N"),
        alt.Tooltip("% Market Cap:Q", format=".2f"),
        alt.Tooltip("Market Cap (Trillion IDR):Q", format=",.2f"),
    ]
).properties(
    title="% Market Cap of Each Sector of the Total IDX Market Cap",
    width=400
)

Notice that I use `:Q` and `:N` behind the column names, these are used to specify the data type, with `Q` stands for quantitative, and `N` stands for nominal.

In [ ]:
total_mc_chart

##### Historical Market Cap

In [ ]:
# Snippet of the data
historical_mc.head()

We'll use [line chart](https://altair-viz.github.io/user_guide/marks/line.html) from altair.

In [ ]:
historical_mc_chart = alt.Chart(historical_mc).mark_line(
    point=True # add individual data points to the line chart
).encode(
    x=alt.X("Quarter:N", axis=alt.Axis(labelAngle=0)), # 0 degree of x-axis label angle
    y=alt.Y("Market Cap (Trillion IDR):Q"),
    color=alt.Color(
        "Sector:N",
        scale=alt.Scale(scheme="lightgreyred"),
        sort=alt.SortField(field="Sector", order="ascending") # sort color based on the sector's name
    ),
    tooltip=[
        alt.Tooltip("Sector:N"),
        alt.Tooltip("Quarter:N"),
        alt.Tooltip("Market Cap (Trillion IDR):Q", format=".2f")
    ]
).properties(
    title="Historical Market Cap Across Sectors",
    width=500
)

In [ ]:
historical_mc_chart

#### Historical Market Cap Change

In [ ]:
# Snippet of the data
change_mc.head()

In [ ]:
change_mc_chart = alt.Chart(change_mc).mark_line(
    point=True # add individual data points to the line chart
).encode(
    x=alt.X("Date:T"),
    y=alt.Y("Market Cap Change (%):Q"),
    color=alt.Color(
        "Sector:N",
        scale=alt.Scale(scheme="lightgreyred"),
        sort=alt.SortField(field="Sector", order="ascending") # sort color based on the sector's name
    ),
    tooltip=[
        alt.Tooltip("Sector:N"),
        alt.Tooltip("Date:T"),
        alt.Tooltip("Market Cap Change (%):Q", format=".2f")
    ]
).properties(
    title="Historical Market Cap Change Across Sectors",
    width=900
)

In [ ]:
change_mc_chart

### Valuation

Now let's move on to the Valuation. There are four valuation metrics that we'll try to compare:
- Price/Book Ratio
- Price/Earning Ratio
- Price/Sales Ratio
- Price/Cash Flow Ratio

Comparing valuation can be useful to evaluate which sectors are overvalued and which are undervalued.

Let's first retrieve the data:

In [ ]:
# Call retrieve_valuation_data() function
df_valuation = retrieve_valuation_data(sectors_list[0:3])

In [ ]:
df_valuation.head()

We'll use the line chart again.

In [ ]:
valuation_chart = alt.Chart(df_valuation).mark_line(
    point=True # add individual data points to the line chart
).encode(
    x=alt.X("Year:N", axis=alt.Axis(labelAngle=0)), # 0 degree of x-axis label angle
    y=alt.Y("Price/Book Ratio:Q"),
    color=alt.Color(
        "Sector:N",
        scale=alt.Scale(scheme="lightgreyred"),
        sort=alt.SortField(field="Sector", order="ascending") # sort color based on the sector's name
    ),
    tooltip=[
        alt.Tooltip("Sector:N"),
        alt.Tooltip("Year:N"),
        alt.Tooltip("Price/Book Ratio:Q", format=".2f")
    ]
).properties(
    title="Price/Book Ratio Across Sectors",
    width=900
)

In [ ]:
valuation_chart

For the other metrics, just change all the `Price/Book Ratio` in the code above with the metric: `Price/Earning Ratio`, `Price/Sales Ratio`, or `Price/Cash Flow Ratio`, and you should get a similar visualization as above but with the chosen metric value.

In [ ]:
# Make the function to be dynamic based on the selected metric
selected_metric = "Price/Earning Ratio"
valuation_chart = alt.Chart(df_valuation).mark_line(
    point=True # add individual data points to the line chart
).encode(
    x=alt.X("Year:N", axis=alt.Axis(labelAngle=0)), # 0 degree of x-axis label angle
    y=alt.Y(f"{selected_metric}:Q"),
    color=alt.Color(
        "Sector:N",
        scale=alt.Scale(scheme="lightgreyred"),
        sort=alt.SortField(field="Sector", order="ascending") # sort color based on the sector's name
    ),
    tooltip=[
        alt.Tooltip("Sector:N"),
        alt.Tooltip("Year:N"),
        alt.Tooltip(f"{selected_metric}:Q", format=".2f")
    ]
).properties(
    title=f"{selected_metric} Across Sectors",
    width=900
)

In [ ]:
valuation_chart

### Top Companies

The last aspect we'll compare between the sectors is the Top Companies. By doing this analysis, we can directly find the top performing companies in the sectors we compare. For this comparison, we'll use four criteria to identify the top companies in each sector:

- Top companies based on Market Cap
- Top companies based on Revenue Growth
- Top companies based on Profit
- Top companies based on Revenue

Let's first retrieve the data:

In [ ]:
# Call retrieve_top_companies_data() function
df_top_mc, df_top_growth, df_top_profit, df_top_revenue = retrieve_top_companies_data(sectors_list[0:3])

#### Top Companies based on the total market cap

In [ ]:
# Snippet of the data
df_top_mc.head()

We'll use [bar chart](https://altair-viz.github.io/user_guide/marks/bar.html).

In [ ]:
mc_chart = alt.Chart(df_top_mc).mark_bar().encode(
    x=alt.X("Market Cap (Trillion IDR):Q"),
    y=alt.Y("Symbol:N", sort="-x"), # sort y-axis based on the value of the x-axis in descending order
    color=alt.Color(
        "Sector:N",
        scale=alt.Scale(scheme="lightgreyred"),
        sort=alt.SortField(field="Sector", order="ascending") # sort color based on the sector's name
    ),
    tooltip=[
        alt.Tooltip("Sector:N"),
        alt.Tooltip("Symbol:N"),
        alt.Tooltip("Market Cap (Trillion IDR):Q", format=".2f")
    ]
).properties(
    title="Top Companies based on Market Cap Across Sectors",
    width=900,
    height=500,
)

In [ ]:
mc_chart

In [ ]:
growth_chart = alt.Chart(df_top_growth).mark_bar().encode(
    x=alt.X("Revenue Growth (%):Q"),
    y=alt.Y("Symbol:N", sort="-x"), # sort y-axis based on the value of the x-axis in descending order
    color=alt.Color(
        "Sector:N",
        scale=alt.Scale(scheme="lightgreyred"),
        sort=alt.SortField(field="Sector", order="ascending") # sort color based on the sector's name
    ),
    tooltip=[
        alt.Tooltip("Sector:N"),
        alt.Tooltip("Symbol:N"),
        alt.Tooltip("Revenue Growth (%):Q", format=",.2f")
    ]
).properties(
    title="Top Companies based on Revenue Growth Across Sectors",
    width=900,
    height=500,
)

In [ ]:
growth_chart

In [ ]:
profit_chart = alt.Chart(df_top_profit).mark_bar().encode(
    x=alt.X("Profit (Billion IDR):Q"),
    y=alt.Y("Symbol:N", sort="-x"), # sort y-axis based on the value of the x-axis in descending order
    color=alt.Color(
        "Sector:N",
        scale=alt.Scale(scheme="lightgreyred"),
        sort=alt.SortField(field="Sector", order="ascending") # sort color based on the sector's name
    ),
    tooltip=[
        alt.Tooltip("Sector:N"),
        alt.Tooltip("Symbol:N"),
        alt.Tooltip("Profit (Billion IDR):Q", format=",.2f")
    ]
).properties(
    title="Top Companies based on Profit Across Sectors",
    width=900,
    height=500,
)

In [ ]:
profit_chart

In [ ]:
revenue_chart = alt.Chart(df_top_revenue).mark_bar().encode(
    x=alt.X("Revenue (Trillion IDR):Q"),
    y=alt.Y("Symbol:N", sort="-x"), # sort y-axis based on the value of the x-axis in descending order
    color=alt.Color(
        "Sector:N",
        scale=alt.Scale(scheme="lightgreyred"),
        sort=alt.SortField(field="Sector", order="ascending") # sort color based on the sector's name
    ),
    tooltip=[
        alt.Tooltip("Sector:N"),
        alt.Tooltip("Symbol:N"),
        alt.Tooltip("Revenue (Trillion IDR):Q", format=",.2f")
    ]
).properties(
    title="Top Companies based on Revenue Across Sectors",
    width=900,
    height=500,
)

In [ ]:
revenue_chart

Now let's move all our analysis to the streamlit app!